### 协同过滤推荐

In [28]:
#构造一份打分数据集，可以去movielens下载真实的数据做实验
users = {"小明": {"中国合伙人": 5.0, "太平轮": 3.0, "荒野猎人": 4.5, "老炮儿": 5.0, "我的少女时代": 3.0, "肖洛特烦恼": 4.5, "火星救援": 5.0},
         "小红":{"小时代4": 4.0, "荒野猎人": 3.0, "我的少女时代": 5.0, "肖洛特烦恼": 5.0, "火星救援": 3.0, "后会无期": 3.0},
         "小阳": {"小时代4": 2.0, "中国合伙人": 5.0, "我的少女时代": 3.0, "老炮儿": 5.0, "肖洛特烦恼": 4.5, "速度与激情7": 5.0},
         "小四": {"小时代4": 5.0, "中国合伙人": 3.0, "我的少女时代": 4.0, "匆匆那年": 4.0, "速度与激情7": 3.5, "火星救援": 3.5, "后会无期": 4.5},
         "六爷": {"小时代4": 2.0, "中国合伙人": 4.0, "荒野猎人": 4.5, "老炮儿": 5.0, "我的少女时代": 2.0, "Phoenix": 4.0, "The Strokes": 4.0},
         "小李":  {"荒野猎人": 5.0, "盗梦空间": 5.0, "我的少女时代": 3.0, "速度与激情7": 5.0, "蚁人": 4.5, "老炮儿": 4.0, "后会无期": 3.5, "匆匆那年": 3.5},
         "隔壁老王": {"荒野猎人": 5.0, "中国合伙人": 4.0, "我的少女时代": 1.0, "Phoenix": 5.0, "甄嬛传": 4.0, "The Strokes": 5.0, "蚁人": 4.0},
         "邻村小芳": {"小时代4": 4.0, "我的少女时代": 4.5, "匆匆那年": 4.5, "甄嬛传": 2.5, "Phoenix": 3.0, "中国合伙人": 3.0},
         "李雷":{"匆匆那年": 3.0,"中国合伙人": 4.0, "火星救援": 5.0, "小时代4": 2.0, "肖洛特烦恼": 4.5, "The Strokes": 4.0},
         "韩梅梅":{"我的少女时代": 5.0,"荒野猎人": 3.0, "盗梦空间": 3.0, "小时代4": 5.0, "匆匆那年": 4.5, "甄嬛传": 4.5}
        }

### 定义距离计算函数
这是一个纯手写版本的距离计算函数集，但是大家可以在scipy当中找到更便捷的库内置计算函数

In [14]:
from math import sqrt

def euclidean_dis(rating1, rating2):
    """计算2个打分序列间的欧式距离. 输入的rating1和rating2都是打分dict
       格式为{'小时代4': 1.0, '疯狂动物城': 5.0}"""
    distance = 0
    commonRatings = False 
    for key in rating1:
        if key in rating2:
            distance += (rating1[key] - rating2[key])^2
            commonRatings = True
    #两个打分序列之间有公共打分电影
    if commonRatings:
        return distance
    #无公共打分电影
    else:
        return -1


def manhattan_dis(rating1, rating2):
    """计算2个打分序列间的曼哈顿距离. 输入的rating1和rating2都是打分dict
       格式为{'小时代4': 1.0, '疯狂动物城': 5.0}"""
    distance = 0
    commonRatings = False 
    for key in rating1:
        if key in rating2:
            distance += abs(rating1[key] - rating2[key])
            commonRatings = True
    #两个打分序列之间有公共打分电影
    if commonRatings:
        return distance
    #无公共打分电影
    else:
        return -1

def cos_dis(rating1, rating2):
    """计算2个打分序列间的cos距离. 输入的rating1和rating2都是打分dict
       格式为{'小时代4': 1.0, '疯狂动物城': 5.0}"""
    distance = 0
    dot_product_1 = 0
    dot_product_2 = 0
    commonRatings = False
    
    for score in rating1.values(): 
        dot_product_1 += score^2
    for score in rating2.values():
        dot_product_2 += score^2
        
    for key in rating1:
        if key in rating2:
            distance += rating1[key] * rating2[key]
            commonRatings = True
    #两个打分序列之间有公共打分电影
    if commonRatings:
        return 1-distance/sqrt(dot_product_1*dot_product_2)
    #无公共打分电影
    else:
        return -1

def pearson_dis(rating1, rating2):
    """计算2个打分序列间的pearson距离. 输入的rating1和rating2都是打分dict
       格式为{'小时代4': 1.0, '疯狂动物城': 5.0}"""
    sum_xy = 0
    sum_x = 0
    sum_y = 0
    sum_x2 = 0
    sum_y2 = 0
    n = 0
    for key in rating1:
        if key in rating2:
            n += 1
            x = rating1[key]
            y = rating2[key]
            sum_xy += x * y
            sum_x += x
            sum_y += y
            sum_x2 += pow(x, 2)
            sum_y2 += pow(y, 2)
    # now compute denominator
    denominator = sqrt((sum_x2 * n - pow(sum_x, 2)) * (sum_y2 * n - pow(sum_y, 2)))
    if denominator:
         return (sum_xy * n - (sum_x * sum_y)) / denominator
    else:
         return 0                       

In [42]:
from collections import OrderedDict
from operator import itemgetter

def computeNearestNeighbor(username, users):
    """在给定username的情况下，计算其他用户和它的距离并排序"""
    distances = {}
    for user in users:
        if user != username:
            distances[user] = pearson_dis(users[user], users[username])
    # 根据相似度排序，相似度越高，排得越靠前
    #distances = OrderedDict(sorted(distances.items(), key = lambda x:x[1], reverse=True))
    return distances

#推荐 nearest neighbor movies
def recommendOne(username, users):
    """对指定的user推荐电影"""
    # 找到最近邻
    nearestNeighbors = computeNearestNeighbor(username, users)
    nearestNeighbor = max(nearestNeighbors.items(), key=itemgetter(1))[0]
    
    recommendations = []
    userRatings = users[username]
    neighborRatings = users[nearestNeighbor]
    # 找到最近邻看过，但是我们没看过的电影，计算推荐
    for title in neighborRatings:
        if not title in userRatings:
            recommendations.append((neighborRatings[title], title))
            
    recommendations.sort(reverse = True)
    for result in recommendations:
        print(result[1], result[0])
        
#推荐 user-based Top K movies
def recommendTopK(username, users, k):
    """对指定的user推荐电影"""
    nearestNeighbors = computeNearestNeighbor(username, users)
    
    weights = {}
    weightedRatings = {}
    userRatings = users[username]
    for neighbor, weight in nearestNeighbors.items():
        if weight > 0:
            neighborRatings = users[neighbor]
            for title, rating in neighborRatings.items():
                if title not in userRatings:
                    if title in weightedRatings:
                        weights[title] += weight
                        weightedRatings[title] += weight*rating
                    else:
                        weights[title] = weight
                        weightedRatings[title] = weight*rating
    
    for w in weightedRatings.keys():
        if w in weights:
            weightedRatings[w] = round(weightedRatings[w]/weights[w],1)
            
    recommendations = sorted(weightedRatings.items(), key=itemgetter(1), reverse=True)[:k]
    for result in recommendations:
        print(result[0],result[1])

In [43]:
recommendOne("小李", users)
print('------------------')
recommendTopK("小李", users, 5)

The Strokes 5.0
Phoenix 5.0
甄嬛传 4.0
中国合伙人 4.0
------------------
火星救援 5.0
Phoenix 4.6
The Strokes 4.6
肖洛特烦恼 4.5
中国合伙人 4.5


In [29]:
movies = {}
for person,ratings in users.items():
    for title,star in ratings.items():
        movies.setdefault(title,{})[person] = star
print(movies)

{'肖洛特烦恼': {'李雷': 4.5, '小红': 5.0, '小阳': 4.5, '小明': 4.5}, '中国合伙人': {'邻村小芳': 3.0, '小明': 5.0, '六爷': 4.0, '李雷': 4.0, '小阳': 5.0, '小四': 3.0, '隔壁老王': 4.0}, '甄嬛传': {'邻村小芳': 2.5, '隔壁老王': 4.0, '韩梅梅': 4.5}, '荒野猎人': {'小明': 4.5, '六爷': 4.5, '韩梅梅': 3.0, '隔壁老王': 5.0, '小红': 3.0, '小李': 5.0}, '速度与激情7': {'小阳': 5.0, '小四': 3.5, '小李': 5.0}, '太平轮': {'小明': 3.0}, 'The Strokes': {'李雷': 4.0, '六爷': 4.0, '隔壁老王': 5.0}, '小时代4': {'邻村小芳': 4.0, '六爷': 2.0, '韩梅梅': 5.0, '李雷': 2.0, '小红': 4.0, '小阳': 2.0, '小四': 5.0}, '老炮儿': {'六爷': 5.0, '小阳': 5.0, '小李': 4.0, '小明': 5.0}, '我的少女时代': {'邻村小芳': 4.5, '小明': 3.0, '六爷': 2.0, '韩梅梅': 5.0, '隔壁老王': 1.0, '小红': 5.0, '小阳': 3.0, '小四': 4.0, '小李': 3.0}, '蚁人': {'隔壁老王': 4.0, '小李': 4.5}, '后会无期': {'小红': 3.0, '小四': 4.5, '小李': 3.5}, '盗梦空间': {'韩梅梅': 3.0, '小李': 5.0}, '匆匆那年': {'邻村小芳': 4.5, '李雷': 3.0, '小四': 4.0, '小李': 3.5, '韩梅梅': 4.5}, 'Phoenix': {'邻村小芳': 3.0, '隔壁老王': 5.0, '六爷': 4.0}, '火星救援': {'李雷': 5.0, '小红': 3.0, '小四': 3.5, '小明': 5.0}}


In [18]:
from collections import OrderedDict
from operator import itemgetter

def computeNearestNeighbor(itemname, items):
    """在给定username的情况下，计算其他用户和它的距离并排序"""
    distances = {}
    for item in items:
        if item != itemname:
            distances[item] = pearson_dis(items[item], items[itemname])      
    # 根据相似度排序，相似度越高，排得越靠前
    #distances = OrderedDict(sorted(distances.items(), key = lambda x:x[1], reverse=True))
    return distances

#推荐 nearest neighbor users
def recommendOne(itemname, items):
    """对指定的电影推荐user"""
    nearestNeighbors = computeNearestNeighbor(itemname, items)
    nearestNeighbor = max(nearestNeighbors.items(), key=itemgetter(1))[0]
    
    recommendations = []
    itemRatings = items[itemname]
    neighborRatings = items[nearestNeighbor]
    for user in neighborRatings:
        if not user in itemRatings:
            recommendations.append((neighborRatings[user], user))
            
    recommendations.sort(reverse = True)
    for result in recommendations:
        print(result[1], result[0])
        
#推荐 item-based Top K users
def recommendTopK(itemname, items, k):
    """对指定的电影推荐user"""
    nearestNeighbors = computeNearestNeighbor(itemname, items)
    
    weights = {}
    weightedRatings = {}
    itemRatings = items[itemname]
    for neighbor, weight in nearestNeighbors.items():
        if weight > 0:
            neighborRatings = items[neighbor]
            for title, rating in neighborRatings.items():
                if title not in itemRatings:
                    if title in weightedRatings:
                        weights[title] += weight
                        weightedRatings[title] += weight*rating
                    else:
                        weights[title] = weight
                        weightedRatings[title] = weight*rating
    
    for w in weightedRatings.keys():
        if w in weights:
            weightedRatings[w] = round(weightedRatings[w]/weights[w],1)
            
    recommendations = sorted(weightedRatings.items(), key=itemgetter(1), reverse=True)[:k]
    for result in recommendations:
        print(result[0], result[1])

In [19]:
recommendOne("速度与激情7", movies)
print('------------------')
recommendTopK("速度与激情7", movies, 5)

小明 5.0
隔壁老王 4.0
李雷 4.0
六爷 4.0
邻村小芳 3.0
------------------
小明 5.0
六爷 4.0
隔壁老王 4.0
李雷 4.0
邻村小芳 3.0


### Latent Factor Model
这里手写了一个矩阵分解来完成一个LFM模型，矩阵分解的过程和PPT中提到的公式是一致的

In [38]:
import numpy as np
from pandas import DataFrame

#读取user数据并用张量分解进行打分
#现在有很多很方便对高维矩阵做分解的package，比如libmf, svdfeature等
def LatentFactorModel(R, K, steps=5000, alpha=0.0002, beta=0.02):
    N = R.shape[0]
    if(K > N):
        print('invalid K')
        return
    
    M = R.shape[1]
    P = np.random.rand(N,K)
    Q = np.random.rand(K,M) 
    
    for step in range(steps):
        for i in range(N):
            for j in range(M):
                if R.iloc[i,j] > 0:
                    eij = R.iloc[i,j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] += alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] += alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        e = 0
        for i in range(N):
            for j in range(M):
                if R.iloc[i,j] > 0:
                    e += pow(R.iloc[i,j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e += (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        if e < 0.001:
            break
            
    nR = DataFrame(data = np.dot(P,Q), columns = R.columns, index = R.index)
    nR[nR - R < 0.5] = 0
    return nR

def RecommendTopK(nR,user,topK):
    recommendations = dict(nR.loc[user])
    recommendations = sorted(recommendations.items(),key=itemgetter(1),reverse=True)[:5]
    for result in recommendations:
        print(result[0], round(result[1], 1))

In [32]:
K = 4
R = DataFrame.from_dict(movies).fillna(0)
nR = LatentFactorModel(R, K)

In [44]:
RecommendTopK(nR,"小李",5)

The Strokes 4.7
肖洛特烦恼 4.5
中国合伙人 4.4
Phoenix 4.3
甄嬛传 4.2
